# Apartado 1 Naive-Bayes propio
• Tabla con los resultados de la ejecución para los conjuntos de datos
analizados (wdbc y heart). Considerar los dos tipos de particionado. Los
resultados se refieren a las tasas de error y deben mostrarse tanto con
la corrección de Laplace como sin ella. Se debe incluir tanto el
promedio de error como su desviación típica. Es importante mostrar
todos los resultados agrupados en una tabla para facilitar su evaluación.

In [1]:
import pandas as pd
import numpy as np
import random
import EstrategiaParticionado
from Datos import Datos
from Clasificador import ClasificadorNaiveBayes
from os import listdir

resultados_nb_propio = []
# Damos un valor aleatorio a la semilla para cada ejecución
seed = random.random()

# Ejecutamos cada dataset que se encuentre en la carpeta Datasets
for archivo in listdir('Datasets/'):
    dataset = Datos('Datasets/' + archivo)
    
    # Parámetros de las estrategias de particionado
    n_ejecuciones = 5
    n_folds = 5
    estrategia_simple = EstrategiaParticionado.ValidacionSimple(n_ejecuciones, 0.25)
    estrategia_cruzada = EstrategiaParticionado.ValidacionCruzada(n_folds)
    nb_simple_laplace = ClasificadorNaiveBayes(laplace=1)
    nb_cruzada_laplace = ClasificadorNaiveBayes(laplace=1)

    # Con corrección de Laplace
    error_simple_laplace = nb_simple_laplace.validacion(estrategia_simple, dataset, nb_simple_laplace, seed)
    error_cruzada_laplace = nb_cruzada_laplace.validacion(estrategia_cruzada, dataset, nb_cruzada_laplace)
    
    # Sin corrección de Laplace
    nb_simple = ClasificadorNaiveBayes(laplace=0)
    nb_cruzada = ClasificadorNaiveBayes(laplace=0)
    error_simple = nb_simple.validacion(estrategia_simple, dataset, nb_simple, seed)
    error_cruzada = nb_cruzada.validacion(estrategia_cruzada, dataset, nb_cruzada)
    
    # Calculamos los promedios y las desviaciones típicas y las almacenamos en los resutlados 
    # para posteriormente mostrarlos en una tabla
    resultados_nb_propio.append({
        'Dataset': archivo,
        'Particionado': 'Simple',
        'Laplace': True,
        'Error Promedio': np.mean(error_simple_laplace),
        'Desviación Típica': np.std(error_simple_laplace)
    })
    resultados_nb_propio.append({
        'Dataset': archivo,
        'Particionado': 'Simple',
        'Laplace': False,
        'Error Promedio': np.mean(error_simple),
        'Desviación Típica': np.std(error_simple)
    })
    resultados_nb_propio.append({
        'Dataset': archivo,
        'Particionado': 'Cruzada',
        'Laplace': True,
        'Error Promedio': np.mean(error_cruzada_laplace),
        'Desviación Típica': np.std(error_cruzada_laplace)
    })
    resultados_nb_propio.append({
        'Dataset': archivo,
        'Particionado': 'Cruzada',
        'Laplace': False,
        'Error Promedio': np.mean(error_cruzada),
        'Desviación Típica': np.std(error_cruzada)
    })

# Convertimos los resultados en un dataframe
df_resultados_nb_propio = pd.DataFrame(resultados_nb_propio)

# Mostramos la tabla
display(df_resultados_nb_propio)

,Dataset,Particionado,Laplace,Error Promedio,Desviación Típica
0,wdbc.csv,Simple,True,0.067606,0.014501
1,wdbc.csv,Simple,False,0.067606,0.014501
2,wdbc.csv,Cruzada,True,0.070315,0.031910
3,wdbc.csv,Cruzada,False,0.070315,0.031910
4,heart.csv,Simple,True,0.146087,0.021050
5,heart.csv,Simple,False,0.146087,0.021050
6,heart.csv,Cruzada,True,0.146110,0.053510
7,heart.csv,Cruzada,False,0.146110,0.053510


• Breve análisis de los resultados anteriores. Discutir el efecto Laplace.

Para la creación de particiones se han empleado parámetros estándar que son los que se encuentran por defecto en las versiones correspondientes de scikit learn, en concreto,
para la validación simple se ha utilizado un porcentaje del 75% (1 - 0.25) para el tamaño de la partición de entrenamiento y para la validación cruzada se han utilizado 5 folds.

Atendiendo a los resultados del conjunto de datos "wdbc.csv" podemos ver que tanto el error promedio como la desviación típica son muy bajos, indicando que la clasificación es muy buena teniendo una tasa de error promedio cercana al 5-7%, esto puede deberse a que los datos continuos siguen realmente una distribución Gaussiana y por lo tanto al suponer nuestro Naive Bayes una distribución Gaussiana para estos datos se obtiene este error tan bajo. Por otro lado, observamos que la corrección Laplaciana no afecta al resultado, esto se explica por el hecho de que el conjunto de datos unicamente cuenta con atributos continuos, y no aplicamos dicha corrección para atributos continuos. 

Analizando los resultados del conjunto "heart.csv" observamos unos ratios de error ligeramente más elevados que para el otro conjunto, esto posiblemente se pueda explicar por el hecho de que alguno de los atributos no siga una distribución Gaussiana y a la combinación de verosimilitudes de atributos continuos con categóricos, sin embargo, el error sigue siendo relativamente bajo rondando un 15%. En cuanto a Laplace para conjuntos de entrenamiento grandes no se observa ninguna diferencia significativa puesto que no hay datos que falten en el entrenamiento y Laplace no tiene efecto, sin embargo, si reducimos significativamente el tamaño del conjunto de entrenamiento podemos ver una diferencia alrededor del 5% entre aplicar el suavizado o no, siendo el NB con suavizado más preciso, también hay que considerar que esto es dependiente del factor de suavizado que se aplique, de tal manera que si faltan muchos datos se puede aplicar un factor de corrección de Laplace mayor para obtener una representación mayor para los atributos faltantes y por lo tanto una mejor clasificación.

# Apartado 2 Naive-Bayes Scikit-Learn
• Tabla de resultados equivalente a la anterior, pero utilizando los
métodos del paquete scikit-learn: MultinomialNB, GaussianNB y
CategoricalNB.

In [2]:
import pandas as pd
import numpy as np
from Datos import Datos
from sklearn import model_selection
from sklearn import naive_bayes as nb

def validacion_simple(nb, test_x, test_y):
    error_promedio = 1 - nb.score(test_x, test_y)
    return error_promedio

def validacion_cruzada(df, nb, target): 
    error_promedio = 1 - model_selection.cross_val_score(nb, df, target, cv=5)
    return error_promedio

resultados_nb_sklearn = []
datasets = ['Datasets/wdbc.csv', 'Datasets/heart.csv']

for dataset in datasets:
    datos = Datos(dataset)
    df = datos.datos

    # Separamos la columna target
    target = df['Class']
    df = df.drop('Class', axis=1)
    
    n_ejecuciones = 5
    lista_errores_simple_laplace_cat = []
    lista_errores_simple_cat = []
    lista_errores_simple_mult = []
    lista_errores_simple_gauss = []

    if dataset == 'Datasets/wdbc.csv':
        # Validación con Naive Bayes Gaussiano
        for i in range(n_ejecuciones):
            train_X, test_X, train_y, test_y = model_selection.train_test_split(
                df, target, test_size=0.25)
            
            nb_gaussian = nb.GaussianNB()
            nb_gaussian.fit(train_X, train_y)
            
            # Validación Simple
            lista_errores_simple_gauss.append(validacion_simple(nb_gaussian, test_X, test_y))
            
        nb_gaussian = nb.GaussianNB()
        error_cruzada_gauss = validacion_cruzada(df, nb_gaussian, target)
        resultados_nb_sklearn.append({
            'Dataset': dataset,
            'Particionado': 'Simple',
            'NB': 'Gaussian',
            'Error Promedio': np.mean(lista_errores_simple_gauss),
            'Desviación Típica': np.std(lista_errores_simple_gauss)
        })
        resultados_nb_sklearn.append({
            'Dataset': dataset,
            'Particionado': 'Cruzada',
            'NB': 'Gaussian',
            'Error Promedio': np.mean(error_cruzada_gauss),
            'Desviación Típica': np.std(error_cruzada_gauss)
        })
    else:
        seed = 10

        cols_num = []
        cols_cat = []
        for idx, col in enumerate(df.columns):
            if datos.nominalAtributos[idx] is False:
                cols_num.append(col)
            else:
                cols_cat.append(col)
        
        lista_errores_simple_clg = []
        lista_errores_simple_cg = []
        lista_errores_simple_mg = []
    
        for _ in range(n_ejecuciones):
            train_X, test_X, train_y, test_y = model_selection.train_test_split(df, target, test_size=0.25)
    
            # Entrenamos modelos Naive Bayes independientes
            nb_gaussian = nb.GaussianNB()
            nb_multinomial = nb.MultinomialNB()
            nb_categorical_laplace = nb.CategoricalNB()
            nb_categorical = nb.CategoricalNB(alpha=0)
    
            nb_gaussian.fit(train_X[cols_num], train_y)
            nb_categorical_laplace.fit(train_X[cols_cat], train_y)
            nb_categorical.fit(train_X[cols_cat], train_y)
            nb_multinomial.fit(train_X[cols_cat], train_y)
    
            # Obtenemos probabilidades de clase
            gaussian_probas = nb_gaussian.predict_proba(test_X[cols_num])
            categorical_laplace_probas = nb_categorical_laplace.predict_proba(test_X[cols_cat])
            categorical_probas = nb_categorical.predict_proba(test_X[cols_cat])
            multinomial_probas = nb_multinomial.predict_proba(test_X[cols_cat])
    
            # Combinamos las probabilidades en una nueva matriz de características
            clg = np.hstack((categorical_laplace_probas, gaussian_probas))
            cg = np.hstack((categorical_probas, gaussian_probas))
            mg = np.hstack((multinomial_probas, gaussian_probas))
    
            # Ajustamos un nuevo modelo Gaussiano sobre las nuevas características combinadas
            nb_clg = nb.GaussianNB()
            nb_cg = nb.GaussianNB()
            nb_mg = nb.GaussianNB()
    
            nb_clg.fit(clg, test_y)
            nb_cg.fit(cg, test_y)
            nb_mg.fit(mg, test_y)
    
            # Validación Simple sobre los modelos combinados
            lista_errores_simple_clg.append(validacion_simple(nb_clg, clg, test_y))
            lista_errores_simple_cg.append(validacion_simple(nb_cg, cg, test_y))
            lista_errores_simple_mg.append(validacion_simple(nb_mg, mg, test_y))

        # Validación cruzada para los modelos combinados
        nb_gaussian = nb.GaussianNB()
        nb_multinomial = nb.MultinomialNB()
        nb_categorical_laplace = nb.CategoricalNB()
        nb_categorical = nb.CategoricalNB(alpha=0)
        
        nb_gaussian.fit(train_X[cols_num], train_y)
        nb_categorical_laplace.fit(train_X[cols_cat], train_y)
        nb_categorical.fit(train_X[cols_cat], train_y)
        nb_multinomial.fit(train_X[cols_cat], train_y)
        
        gaussian_probas = nb_gaussian.predict_proba(df[cols_num])
        categorical_probas = nb_categorical.predict_proba(df[cols_cat])
        multinomial_probas = nb_multinomial.predict_proba(df[cols_cat])

        clg = np.hstack((categorical_probas, gaussian_probas))
        cg = np.hstack((categorical_probas, gaussian_probas))
        mg = np.hstack((multinomial_probas, gaussian_probas))

        error_cruzada_clg = validacion_cruzada(clg, nb.GaussianNB(), target)
        error_cruzada_cg = validacion_cruzada(cg, nb.GaussianNB(), target)
        error_cruzada_mg = validacion_cruzada(mg, nb.GaussianNB(), target)

        resultados_nb_sklearn.append({
            'Dataset': dataset,
            'Particionado': 'Simple',
            'NB': 'Categorical + Gaussian',
            'Laplace': True,
            'Error Promedio': np.mean(lista_errores_simple_clg),
            'Desviación Típica': np.std(lista_errores_simple_clg)
        })
        resultados_nb_sklearn.append({
            'Dataset': dataset,
            'Particionado': 'Simple',
            'NB': 'Categorical + Gaussian',
            'Laplace': False,
            'Error Promedio': np.mean(lista_errores_simple_cg),
            'Desviación Típica': np.std(lista_errores_simple_cg)
        })
        resultados_nb_sklearn.append({
            'Dataset': dataset,
            'Particionado': 'Simple',
            'NB': 'Multinomial + Gaussian',
            'Error Promedio': np.mean(lista_errores_simple_mg),
            'Desviación Típica': np.std(lista_errores_simple_mg)
        })
        resultados_nb_sklearn.append({
            'Dataset': dataset,
            'Particionado': 'Cruzada',
            'NB': 'Categorical + Gaussian',
            'Laplace': True,
            'Error Promedio': np.mean(error_cruzada_clg),
            'Desviación Típica': np.std(error_cruzada_clg)
        })
        resultados_nb_sklearn.append({
            'Dataset': dataset,
            'Particionado': 'Cruzada',
            'NB': 'Categorical + Gaussian',
            'Laplace': False,
            'Error Promedio': np.mean(error_cruzada_cg),
            'Desviación Típica': np.std(error_cruzada_cg)
        })
        resultados_nb_sklearn.append({
            'Dataset': dataset,
            'Particionado': 'Cruzada',
            'NB': 'Multinomial + Gaussian',
            'Error Promedio': np.mean(error_cruzada_mg),
            'Desviación Típica': np.std(error_cruzada_mg)
        })

# Convertimos los resultados en un dataframe
df_resultados_nb_sklearn = pd.DataFrame(resultados_nb_sklearn)

# Mostramos la tabla
display(df_resultados_nb_sklearn)

,Dataset,Particionado,NB,Error Promedio,Desviación Típica,Laplace
0,Datasets/wdbc.csv,Simple,Gaussian,0.067133,0.023652,NaN
1,Datasets/wdbc.csv,Cruzada,Gaussian,0.061481,0.014586,NaN
2,Datasets/heart.csv,Simple,Categorical + Gaussian,0.130435,0.020761,True
3,Datasets/heart.csv,Simple,Categorical + Gaussian,0.129565,0.022238,False
4,Datasets/heart.csv,Simple,Multinomial + Gaussian,0.154783,0.022609,NaN
5,Datasets/heart.csv,Cruzada,Categorical + Gaussian,0.151485,0.034710,True
6,Datasets/heart.csv,Cruzada,Categorical + Gaussian,0.151485,0.034710,False
7,Datasets/heart.csv,Cruzada,Multinomial + Gaussian,0.174311,0.042348,NaN


• ¿Existe algún problema con alguno de estos métodos en alguno de los
dos ficheros? En caso afirmativo, ¿en cuál? ¿por qué? ¿cómo podría
resolverse?

En "wdbc.csv" tanto MultinomialNB como CategoricalNB presentan problemas porque el conjunto de datos contiene solo valores numéricos continuos, y estos modelos están diseñados para tratar unicamente con datos categóricos, con lo cual, la solución adecuada para "wdbc.csv" pasa por usar GaussianNB, el cual está diseñado para datos numéricos continuos, o discretizar las variables continuas, en este caso hemos optado por emplear GaussianNB si no existen datos categóricos, como es el caso de dicho conjunto. 

En "heart.csv" hay datos mixtos, es decir, numéricos y categóricos, por ende MultinomialNB y CategoricalNB tendrán problemas para los atributos continuos y GaussianNB para los atributos discretos o categóricos, la solución por la que se ha optado es ajustar de manera independiente un modelo GaussianNB para los atributos continuos y un modelo MultinomialNB o CategoricalNB para los atributos categóricos, luego, se transforman las características del conjunto de datos en probabilidades de asignación de clase y se combinan, permitiendo así que un nuevo modelo GaussianNB que captura la información tanto de las características continuas como categóricas en una representación conjunta.

# Apartado 3 K-NN propio
Resultados en forma de tabla de la clasificación mediante
vecinos próximos para los diferentes valores de vecindad en
los conjuntos de datos propuestos. Obtener los resultados
tanto para datos estandarizados como sin estandarizar, con el
objetivo de justificar el rendimiento del algoritmo en base a
estas características. Separar por tipo de validación (simple,
cruzada)

In [3]:
import numpy as np
import pandas as pd
from Datos import Datos
import EstrategiaParticionado
from Clasificador import ClasificadorKNN
from os import listdir

# Valores de K a probar
K_values = [1, 5, 11, 21]

normalizations = [False, True]

# Número de ejecuciones y folds
n_ejecuciones = 5
n_folds = 5

resultados_knn_propio = []
datasets = ['Datasets/wdbc.csv', 'Datasets/heart.csv']

for dataset in datasets:
    df = Datos(dataset)

    for normalizado in normalizations:

        estrategia_simple = EstrategiaParticionado.ValidacionSimple(
            n_ejecuciones, 0.25)
        estrategia_cruzada = EstrategiaParticionado.ValidacionCruzada(n_folds)

        for K in K_values:
            # Validación Simple
            neigh_simple = ClasificadorKNN(K=K, normalize=normalizado)
            errores_simple = neigh_simple.validacion(
                estrategia_simple, df, neigh_simple)

            # Validación Cruzada
            neigh_cruzada = ClasificadorKNN(K=K, normalize=normalizado)
            errores_cruzada = neigh_cruzada.validacion(
                estrategia_cruzada, df, neigh_cruzada)

            resultados_knn_propio.append({
                'Dataset': dataset,
                'Particionado': 'Simple',
                'K': K,
                'Normalizado': normalizado,
                'Error Promedio': np.mean(errores_simple),
                'Desviación Típica': np.std(errores_simple)
            })
            resultados_knn_propio.append({
                'Dataset': dataset,
                'Particionado': 'Cruzada',
                'K': K,
                'Normalizado': normalizado,
                'Error Promedio': np.mean(errores_cruzada),
                'Desviación Típica': np.std(errores_cruzada)
            })

df_resultados_knn_propio = pd.DataFrame(resultados_knn_propio)

display(df_resultados_knn_propio)

,Dataset,Particionado,K,Normalizado,Error Promedio,Desviación Típica
0,Datasets/wdbc.csv,Simple,1,False,0.077465,0.019414
1,Datasets/wdbc.csv,Cruzada,1,False,0.093122,0.034412
2,Datasets/wdbc.csv,Simple,5,False,0.076056,0.015684
3,Datasets/wdbc.csv,Cruzada,5,False,0.073793,0.036174
4,Datasets/wdbc.csv,Simple,11,False,0.073239,0.019209
5,Datasets/wdbc.csv,Cruzada,11,False,0.072023,0.045168
6,Datasets/wdbc.csv,Simple,21,False,0.077465,0.022270
7,Datasets/wdbc.csv,Cruzada,21,False,0.079025,0.062478
8,Datasets/wdbc.csv,Simple,1,True,0.042254,0.014772
9,Datasets/wdbc.csv,Cruzada,1,True,0.042198,0.010327


• Análisis de resultados, del impacto de K y de la estandarización

¿Qué impacto ha tenido la Normalización?

En el primer conjunto de datos, que consta de datos categóricos y numéricos, hemos podido observar una inmensa mejora en elr endimiento tras la aplicación de la estandarización. Sobre todo, podemos ver cómo el error medio disminuye en situaciones en las que el valor de la 'K' es bajo(K=1, K=5). Esta mejora se debe a que al normalizar, todas las características se ajustan a una escala común, además el cálculo de distancias se ve menos afectado por las diferencias de magnitud entre atributos. Para el segundo dataset, compuesto únicamente por datos numéricos continuos, observamos que la aplicación de la normalización provoca una mejor de calidad al evaluar las distancias entre instancias y por tanto, logra una mejora consistente en la precisión para todos los valores de K.

Podemos concluir con que la normalización es fundamental para el rendimiento de nuestro modelo, especialmente en datasets con tipos de datos mixtos, como en heart.csv. La normalización nos otorga la capacidad de comparar de manera más justa entre atributos al reducir las diferencias más significantes por la magnitud.

¿Qué impacto ha tenido el valor de la K?

Para valores pequeños, como K=1 o K=5, observamos que se presenta la mayor cantidad de error en ambos datasets. Esto se debe a que para valores bajos de 'K', el modelo procede a ser más sensible al ruido y a valores atípicos. Con un 'K' pequeño, el modelo es muy sensible a pequeñas variaciones en los datos, ya que un ligero cambio en donde se encuentran los puntos, puede provcar una alteración en el vecino más próximo, cambiando la predicción de clase. En cambio, con valores más altos de K, la decisión se basa en una mayor cantidad de datos, haciendo que el modelo sea menos sensible a pequeñas perturbaciones.

¿Qué impacto han tenido los tipos de validaciones?

Podemos observar que al aplicar la Validación cruzada, no solo obteníamos una ligera disminución en el error promedio, si no que también obteníamos una desviación típica mucho más alta que en validación simple. La mejora en cuanto al error se debe a que se captura la mejor represnetatividad del modelo, esto es, al promediar el error de múltiples particiones, se obtiene una estimación más precisa del error general del modelo en comparación con la validación simple, ya que se basa en una evaluación sobre múltiples combinaciones de los datos.

En cuanto al aumento en la desviación, debido a que el modelo se entrena con un subconjutno de daros ligeramente diferente en cada partición, provocamos que los conjuntos de prueba y entrenamiento estén constantemente cambiando, lo que produce una mayor variabilidad en los resultados de cada partición.


# Apartado 4K-NN Scikit-Learn
• Tabla de resultados equivalente a la del apartado anterior para las
ejecuciones realizadas con la librería KNeighborsClassifier en los
mismos valores de K, datos estandarizados y no estandarizados.

In [4]:
import numpy as np
import pandas as pd
from Datos import Datos
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from sklearn import neighbors as knn
from os import listdir

# Valores de K a probar
K_values = [1, 5, 11, 21]

normalizations = [False, True]

# Número de ejecuciones y folds
n_ejecuciones = 5

resultados_knn_sklearn = []
datasets = ['Datasets/wdbc.csv', 'Datasets/heart.csv']

for dataset in datasets:
    datos = Datos(dataset)
    df = datos.datos
    # Separamos la columna target
    target = df['Class']
    df = df.drop('Class', axis=1)
    
    # Separamos los datos numéricos
    datos_numericos = df.select_dtypes(include='number').columns
    # Aplicamos la estandarización
    dataset_estandarizado = df.copy()
    scaler = StandardScaler()
    dataset_estandarizado[datos_numericos] = scaler.fit_transform(df[datos_numericos])
    
    for normalizado in normalizations:
        # Usamos los datos estandarizados o sin normalizar según el caso
        data = dataset_estandarizado if normalizado else df

        for K in K_values:
            neigh_simple = knn.KNeighborsClassifier(n_neighbors=K, metric='minkowski', p=2)
            neigh_cruzada = knn.KNeighborsClassifier(n_neighbors=K, metric='minkowski', p=2)
            knn_error_simple = []
            
            for _ in range(n_ejecuciones):
                # Realizamos la validación simple y el entrenamiento
                train_X, test_X, train_y, test_y = model_selection.train_test_split(
                    data, target, test_size=0.25)
                neigh_simple.fit(train_X, train_y)
                knn_error_simple.append(1 - neigh_simple.score(test_X, test_y))

            # Validación cruzada
            knn_error_cruzada = 1 - model_selection.cross_val_score(neigh_cruzada, data, target, cv=5)
            
            resultados_knn_sklearn.append({
                'Dataset': dataset,
                'Particionado': 'Simple',
                'K': K,
                'Normalizado': normalizado,
                'Error Promedio': np.mean(knn_error_simple),
                'Desviación Típica': np.std(knn_error_simple)
            })
            resultados_knn_sklearn.append({
                'Dataset': dataset,
                'Particionado': 'Cruzada',
                'K': K,
                'Normalizado': normalizado,
                'Error Promedio': np.mean(knn_error_cruzada),
                'Desviación Típica': np.std(knn_error_cruzada)
            })

df_resultados_knn_sklearn = pd.DataFrame(resultados_knn_sklearn)

display(df_resultados_knn_sklearn)

,Dataset,Particionado,K,Normalizado,Error Promedio,Desviación Típica
0,Datasets/wdbc.csv,Simple,1,False,0.088112,0.016899
1,Datasets/wdbc.csv,Cruzada,1,False,0.094892,0.023754
2,Datasets/wdbc.csv,Simple,5,False,0.074126,0.021936
3,Datasets/wdbc.csv,Cruzada,5,False,0.072054,0.021763
4,Datasets/wdbc.csv,Simple,11,False,0.082517,0.010278
5,Datasets/wdbc.csv,Cruzada,11,False,0.070300,0.027740
6,Datasets/wdbc.csv,Simple,21,False,0.075524,0.019480
7,Datasets/wdbc.csv,Cruzada,21,False,0.070238,0.034136
8,Datasets/wdbc.csv,Simple,1,True,0.050350,0.012818
9,Datasets/wdbc.csv,Cruzada,1,True,0.049231,0.016353


• Análisis de resultados, del impacto de K y de la estandarización

¿Qué impacto ha tenido la Estandarización?

En el conjunto de datos que contiene únicamente datos numéricos continuos, observamos una mejora significativa en la precisión del modelo al aplicar la estandarización. Podemos ver la diferencia en valores de 'K' pequeños como 1 y 5, el error promedio disminuye de manera significativa, de aproximadamente 0.08 y 0.05 a valores de 0.04 y 0.03 respectivamente. Esto se debe a que la estandarización ajusta todas las características a una escala común, eliminando los sesgos que podrían haber afectado el cálculo de distancias debido a la magnitud de los valores.

En el conjunto de datos mixto, podemos observar una disminución abstante evidente en el error promedio al normalizar los datos. Es decir, para K=1, el error promedio baja de aproximadamente 0.36 a 0.18, mientras que para valores de K más altos como K=11 y K=21, el error disminuye de aproximadamente 0.29 a 0.13. 

Esta gran diferencia nos indica que la estandarización es enormemente benificiosa para estos tipos de datasets que contienen datos mixtos, ya que permite que el modelo evalúe las distancias de forma equilibrada entre las distintas características.

¿Qué impacto ha tenido el valor de la K?

Para valores de K pequeños, como K=1 y K=5, observamos un mayor error en ambos conjuntos de datos (wdbc.csv y heart.csv). Esto se debe a que, con valores bajos de K, el modelo es más sensible al ruido y a los valores atípicos. Cuando K es pequeño, el algoritmo se basa en unos pocos vecinos cercanos, lo que puede llevar a errores si esos vecinos son puntos atípicos o de clases minoritarias en la proximidad. En cambio, con valores más altos de K, como K=21, la precisión se estabiliza, especialmente en heart.csv, donde el error promedio baja a aproximadamente 0.13. Esto sugiere que valores de K más altos ayudan a suavizar las predicciones al basarse en un grupo más grande de vecinos, lo que reduce la sensibilidad a pequeñas variaciones en los datos.

# Apartado 5 Conclusión
Comparar y analizar a modo de resumen los resultados propios con
los de Scikit-Learn para los dos algoritmos de aprendizaje y ambos
conjuntos. Utiliza una tabla para comparar los resultados

In [8]:
display(df_resultados_nb_propio)
display(df_resultados_nb_sklearn)

,Dataset,Particionado,Laplace,Error Promedio,Desviación Típica
0,wdbc.csv,Simple,True,0.067606,0.014501
1,wdbc.csv,Simple,False,0.067606,0.014501
2,wdbc.csv,Cruzada,True,0.070315,0.031910
3,wdbc.csv,Cruzada,False,0.070315,0.031910
4,heart.csv,Simple,True,0.146087,0.021050
5,heart.csv,Simple,False,0.146087,0.021050
6,heart.csv,Cruzada,True,0.146110,0.053510
7,heart.csv,Cruzada,False,0.146110,0.053510


,Dataset,Particionado,NB,Error Promedio,Desviación Típica,Laplace
0,Datasets/wdbc.csv,Simple,Gaussian,0.067133,0.023652,NaN
1,Datasets/wdbc.csv,Cruzada,Gaussian,0.061481,0.014586,NaN
2,Datasets/heart.csv,Simple,Categorical + Gaussian,0.130435,0.020761,True
3,Datasets/heart.csv,Simple,Categorical + Gaussian,0.129565,0.022238,False
4,Datasets/heart.csv,Simple,Multinomial + Gaussian,0.154783,0.022609,NaN
5,Datasets/heart.csv,Cruzada,Categorical + Gaussian,0.151485,0.034710,True
6,Datasets/heart.csv,Cruzada,Categorical + Gaussian,0.151485,0.034710,False
7,Datasets/heart.csv,Cruzada,Multinomial + Gaussian,0.174311,0.042348,NaN


En la comparación entre los resultados propios y los obtenidos con sklearn para el algoritmo Naive Bayes, se observa una consistencia en los errores promedio y la desviación típica en el conjunto "wdbc.csv" con un rendimiento casi idéntico en las dos implementaciones, esto tiene sentido ya que en sklearn se emplea el modelo GaussianNB y en nuestra implementación solo se utiliza la distribución gaussiana para calcular la verosimilitud de los datos al no haber datos categóricos, con lo cual en ambas situaciones se emplea la gaussiana de los datos y esto resulta en el rendimiento casi idéntico observado, además la corrección de Laplace no afecta al tratarse de datos continuos. 

En "heart.csv", que incluye datos mixtos, los resultados muestran una ligera ventaja (entre 1% y 2%) en la implementación de sklearn al usar enfoques combinados como Categorical + Gaussian, logrando un error promedio menor.

In [9]:
display(df_resultados_knn_propio)
display(df_resultados_knn_sklearn)

,Dataset,Particionado,K,Normalizado,Error Promedio,Desviación Típica
0,Datasets/wdbc.csv,Simple,1,False,0.077465,0.019414
1,Datasets/wdbc.csv,Cruzada,1,False,0.093122,0.034412
2,Datasets/wdbc.csv,Simple,5,False,0.076056,0.015684
3,Datasets/wdbc.csv,Cruzada,5,False,0.073793,0.036174
4,Datasets/wdbc.csv,Simple,11,False,0.073239,0.019209
5,Datasets/wdbc.csv,Cruzada,11,False,0.072023,0.045168
6,Datasets/wdbc.csv,Simple,21,False,0.077465,0.022270
7,Datasets/wdbc.csv,Cruzada,21,False,0.079025,0.062478
8,Datasets/wdbc.csv,Simple,1,True,0.042254,0.014772
9,Datasets/wdbc.csv,Cruzada,1,True,0.042198,0.010327


,Dataset,Particionado,K,Normalizado,Error Promedio,Desviación Típica
0,Datasets/wdbc.csv,Simple,1,False,0.088112,0.016899
1,Datasets/wdbc.csv,Cruzada,1,False,0.094892,0.023754
2,Datasets/wdbc.csv,Simple,5,False,0.074126,0.021936
3,Datasets/wdbc.csv,Cruzada,5,False,0.072054,0.021763
4,Datasets/wdbc.csv,Simple,11,False,0.082517,0.010278
5,Datasets/wdbc.csv,Cruzada,11,False,0.070300,0.027740
6,Datasets/wdbc.csv,Simple,21,False,0.075524,0.019480
7,Datasets/wdbc.csv,Cruzada,21,False,0.070238,0.034136
8,Datasets/wdbc.csv,Simple,1,True,0.050350,0.012818
9,Datasets/wdbc.csv,Cruzada,1,True,0.049231,0.016353


En la comparación de los resultados entre la implementación propia de KNN y la de sklearn, se observan diferencias notables en los errores promedio y desviaciones típicas, especialmente en el conjunto "heart.csv". En el conjunto "wdbc.csv", que contiene datos continuos, ambas implementaciones ofrecen resultados similares sin normalización, aunque la implementación propia muestra una mayor variabilidad (desviación típica más alta) en algunos valores de K, posiblemente debido a diferencias en la gestión de distancias.

Con la normalización activada, sklearn y nuestra implementación reducen de forma más consistente el error promedio y la desviación típica en ambos conjuntos, indicando una optimización más robusta en el escalado de los datos. Esto es especialmente evidente en "heart.csv", que incluye atributos mixtos. En este conjunto, la implementación propia muestra errores promedio más altos y una mayor variabilidad en particionado cruzado, sugiriendo que sklearn maneja mejor la combinación de atributos de diferentes tipos,